In [17]:
!pip install -q -U sentence-transformers
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 

In [18]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ['안녕하세요?', '조용식입니다.']
embeddings = model.encode(sentences)
print(embeddings)

[[-0.3751044  -0.7733841   0.5927712  ...  0.5792351   0.32683453
  -0.65089685]
 [-0.04237332  0.07960197  0.12686072 ...  0.04116595  0.08316224
   0.22304018]]


# Google Drive 연동

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df = pd.read_excel('/content/drive/MyDrive/SentenceBERT/twitter_chat.xlsx')

In [21]:
df = df.iloc[: , :2]
df.columns = ['question', 'answer']
df.head(2)

,question,answer
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.
1,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예쁘네요.


In [22]:
# 현재 결측치는 없는 것으로 확인
df.isnull().sum() 

# 있을 경우, 
# df = df[~df['q or a'].isna()]

question    0
answer      0
dtype: int64

# Embedding Start

In [25]:
df['embedding'] = pd.Series([[]] * len(df))

df['embedding'] = df['question'].map(lambda x : list(model.encode(x)))
df.head()

,question,answer,embedding
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,"[-0.32713625, -0.21445674, -0.13982062, -0.567..."
1,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예쁘네요.,"[-0.011863271, -0.15321921, -0.47954375, 0.038..."
2,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예뻐요.,"[-0.011863271, -0.15321921, -0.47954375, 0.038..."
3,목 캔디 얼마죠?,700원이요.,"[0.3051233, -0.58904046, 0.6626809, -0.8754816..."
4,수요미식회 민호 실화에요?,다음 주요?,"[-0.04259647, -0.58229643, 0.5716064, -0.66037..."


In [26]:
# 현재 저장 
df.to_csv('twitter_dataset.csv', index=False)

# 간단한 챗봇 테스트

In [34]:
text = '안녕하세요. 조용식입니다. 반갑습니다.'
embedding = model.encode(text)

df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()

,question,answer,embedding,distance
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,"[-0.32713625, -0.21445674, -0.13982062, -0.567...",0.209615
1,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예쁘네요.,"[-0.011863271, -0.15321921, -0.47954375, 0.038...",0.134789
2,사실 여러 가지 글씨체가 있긴 한데 요즘 글씨를 안 쓰다 보니까 너무 맛 가버렸어요...,글씨 예뻐요.,"[-0.011863271, -0.15321921, -0.47954375, 0.038...",0.134789
3,목 캔디 얼마죠?,700원이요.,"[0.3051233, -0.58904046, 0.6626809, -0.8754816...",0.178424
4,수요미식회 민호 실화에요?,다음 주요?,"[-0.04259647, -0.58229643, 0.5716064, -0.66037...",0.213319


In [35]:
answer = df.loc[df['distance'].idxmax()]

print('유사한 질문 :', answer['question'])
print('챗봇 답변 : ', answer['answer'])
print('코사인 유사도 : ', answer['distance'])

유사한 질문 : 안녕하세요.
챗봇 답변 :  좋은 아침인가요?
코사인 유사도 :  0.5917683839797974


# 아래는 GUI / streamlit 
> Python 3.8 이상부터 실행가능한 GUI

In [37]:
import streamlit as st
from streamlit_chat import message
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json

@st.cache(allow_output_mutation=True)
def cached_model():
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    return model

@st.cache(allow_output_mutation=True)
def get_dataset():
    df = pd.read_csv('wellness_dataset.csv')
    df['embedding'] = df['embedding'].apply(json.loads)
    return df

model = cached_model()
df = get_dataset()


# 출처 
st.header('심리상담 챗봇')
st.markdown("[❤️빵형의 개발도상국](https://www.youtube.com/c/빵형의개발도상국)")

if 'generated' not in st.session_state:
    st.session_state['generated'] = []

if 'past' not in st.session_state:
    st.session_state['past'] = []

with st.form('form', clear_on_submit=True):
    user_input = st.text_input('당신: ', '')
    submitted = st.form_submit_button('전송')

if submitted and user_input:
    embedding = model.encode(user_input)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    st.session_state.past.append(user_input)
    st.session_state.generated.append(answer['챗봇'])

for i in range(len(st.session_state['past'])):
    message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')
    if len(st.session_state['generated']) > i:
        message(st.session_state['generated'][i], key=str(i) + '_bot')

ModuleNotFoundError: ignored